In [143]:
import pandas as pd
import pymorphy2
#from nltk.corpus import stopwords
from collections import Counter
import csv

In [107]:
df = pd.read_csv('./data/covid_tweets.csv.gz', compression='gzip')

In [109]:
# размеры данных
df.shape

(1491177, 3)

In [136]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text: str) -> list:
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

In [190]:
# получаем список слов
res = []
for text in df['text'].head(15):
    temp_list_of_words = lemmatize(text)
    for word in temp_list_of_words:
        res.append(word)

In [172]:
# получаем список стоп-слов
with open('./data/stopwords-ru.txt', 'r', encoding='utf-8') as f:
    stopwords = [str(i) for i in f.read().split()]

In [173]:
# удаляем стоп-слова если таковые есть
for word in res:
    if word in stopwords:
        res.remove(word)

In [174]:
for word in res:
    if res.count(word) < 5:
        res.remove(word)

In [193]:
# словарь с указанием сколько раз встречается каждое слово
words_counter = Counter()
for word in res:
    words_counter[word] += 1


In [196]:
with open('word_frequency.csv','w', encoding='utf-8') as csvfile:
    columns = ['word', 'frequency']
    writer=csv.writer(csvfile)
    writer.writerow(columns)
    writer.writerows(words_counter.most_common())